# Meetup Project


What is in a Meetup API request?

1. https://api.meetup.com: The API host.
2. /find/groups: The method
3. zip=11211&radius=1&category=25: The args. Here is where you may add the API key.
    From what I can tell, the only args you can give are zip, radius, and category. So maybe you need to go through the zips 
4. &order=members:  How to order the results.


-  Private groups I cannot see the members who go, only the number. We will need to see how many groups are public.


# Imports / Pymongo Setup

In [68]:
from pymongo import MongoClient
import numpy as np
import requests
import json
key = open('api.txt', 'r').read()

client = MongoClient('localhost', 27017)
db = client.meet_up
collection = db.groups

## Get number of groups in each category globally

In [70]:
# This creates a list of dictionaries, with name, sort_name, id, and shortname. 
dict_categories = json.loads(requests.get('https://api.meetup.com/2/categories?key={}'.format(key)).text)['results']

#make list of ids 
ids = [item['id'] for item in dict_categories]
ids.sort()

In [71]:
def get_count_of_category(number):
    req_count = requests.get('https://api.meetup.com/find/groups?key={}&radius=global&category={}'.format(key, number))\
                        .headers['X-Total-Count']
    cat_name = next(item['name'] for item in dict_categories if item["id"] == number)
    return [number, int(req_count), cat_name]
    
groups_per_category = [get_count_of_category(category) for category in ids]

In [72]:
sum([val[1] for val in groups_per_category])

344914

I find 344,607 groups globally. Wow!

However, I can't seem to get the same number through the json loader. 

Trying with a low group category, Sci-Fi and Fantasy. 

I need to use the offset and page parameter. Thr
Goal for the day: for category 24, get all of its data into mongo. And make experimental design. 

In [115]:
#Meetup only allows 200 requests at a time.
def get_groups_for_category(number):
    number_of_groups = [entry[1] for entry in groups_per_category if entry[0] == number] 
    offsets = int(number_of_groups[0]/200)
    for offset in range(offsets):
        req = requests.get('https://api.meetup.com/find/groups?key={}\
                            &category={}&radius=global&\
                            &offset={}&page=200'.format(key, number, offset))
        json_data = json.loads(req.text)
        collection.insert_many(json_data)

get_groups_for_category(24)